In [1]:
# conda install -c conda-forge pypdf2
# conda install nltk 
# pip install HanTa

import re, nltk, os, glob, docx, json, gensim
import pandas as pd
import numpy as np
import seaborn as sns
import wehs_helpers as wh
# from HanTa import HanoverTagger as ht
from docx2python import docx2python
import win32com.client
from bs4 import BeautifulSoup as bs
import gensim.corpora as corpora
import spacy

# import stopwords
# nltk.download('stopwords')  
os.name  # nt means windows
pd.options.display.max_colwidth = 1000
nlp = spacy.load("de_core_news_lg")
pd.set_option('display.max_rows', 500)
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer 
stemmer = SnowballStemmer("german")
import pickle
pd.options.display.max_colwidth = 1000

In [2]:
#load all reports
rbs=pd.read_pickle('D:/phd/daten/python/db/rechenschaftsberichte_alles.pkl')
#print list of columns
# print(rbs.columns)

df=rbs[['id_fall', 'fall_source_key', 'id_dokument', 'id_k', 'text', 'name','vorname' ,
         'name_mutter', 'vorname_mutter', 'name_vater', 'vorname_vater',  ]]
# die doppelten id_dokumente sind verschiedene Dokumente von verschiedenen Personen
# df2 = df[df.id_dokument.duplicated(keep=False)].sort_values("id_dokument")
# print(df2)

# Prep Violence Stems

# Extraction Text Sections

In [ ]:
# print(stemmer.stem('berichten'))

In [5]:

class ViolenceFinder3:
    # RE_POS_STEMS = ['polizei']
    RE_POS_STEMS = ['gewalt','schl\wg','übergriff']
    
    FALSE_POS_STEMS = ['schlagmüller','ausschlag','einschlag','eingeschlag','niederschlag','niedergeschlag',
                      'schicksalsschlag','angeschlag','vorschlag','vorgeschlag','vorschläge','voranschlag',
                      'ratschlag','ratschläge','schlagzeug','schleg']
    CUT_PHRASES = ['(Welche Lösungen bzw. welches Vorgehen schlägt das Kind bzw. der/die Jugendliche vor)']
    SPLITCHAR = '.'
    SPLITPHRASES = ['Ausgangslage, Auftrag und Ziele\n','Situation, Entwicklung\n','Auftrag und Ziele\n',
                   'Vorgeschichte\n']
    SPLITPATTERN = r'((?<!(z\.B|.\s\w|.\W\w|..\d|Art|bzw|geb|Abs|Stv|etc))\.|;|:|\n\w\)|--|\n\n|\d*\.\d*\.\d\d\d\d\n|\{})'.format('|'.join(SPLITPHRASES))     # three characte expressions
    
    def __init__(self, text):
        self.text = text
        self.sentences = [s for s in re.split(self.SPLITPATTERN, text) if s is not None]
        self.pos_words = [self._find_pos_words(s) for s in self.sentences]
        self.labels = [len(words)>0 for words in self.pos_words]
                
    def _find_pos_words(self,text):
        for phrase in self.CUT_PHRASES:
            text = text.lower().replace(phrase.lower(),'')
        pattern = [r'\b\w*{}\w*\b'.format(stem) for stem in self.RE_POS_STEMS]
        
        pattern = r'(' + r'|'.join(pattern) + r')'
        
        pos_words = []
        for word in re.findall(pattern, text.lower()):
            if not np.any([fpword in word for fpword in self.FALSE_POS_STEMS]):
                pos_words.append(word)
        return pos_words
    
    def label(self):
        return np.any(self.labels)
        
    def pos_sentences(self):
        out = []
        words0 = []
        for ind in np.where(self.labels)[0]:
            out.append(self.sentences[ind].replace('\n','').strip()+self.SPLITCHAR)
            words0.append(self.pos_words[ind])
        
        out3 = []
        out5 = []
        
        for ind in np.where(self.labels)[0]:
            try:
                tempM1 = (self.sentences[ind-2].replace('\n','').strip()+self.SPLITCHAR+' ')
            except IndexError:
                tempM1 = self.SPLITCHAR
            
            temp = (self.sentences[ind].replace('\n','').strip()+self.SPLITCHAR+' ')
            
            try:
                tempP1 =(self.sentences[ind+2].replace('\n','').strip()+self.SPLITCHAR)
            except IndexError:
                tempP1 = self.SPLITCHAR
            
            try:
                tempM2 = (self.sentences[ind-4].replace('\n','').strip()+self.SPLITCHAR+' ')
            except IndexError:
                tempM2 = self.SPLITCHAR
            try:
                tempP2 =(self.sentences[ind+4].replace('\n','').strip()+self.SPLITCHAR)
            except IndexError:
                tempP2 = self.SPLITCHAR 
                
            out3_temp = tempM1 + temp + tempP1
            out3.append(out3_temp)
        #   print(out3_temp)
            out5_temp = tempM2 + tempM1 + temp + tempP1 + tempP2
            out5.append(out5_temp)
            
        return words0, out, out3, out5
        
    
    def marked_text(self, ansi='\033[44;33m', nul = "\033[m"):
        text = ''
        for ii, s in enumerate(self.sentences):
            if self.labels[ii]:
                text += ansi + s + nul + self.SPLITCHAR
            else:
                text += s + self.SPLITCHAR
        return text
    
    def pprint(self):
        print(self.marked_text())



In [6]:

sentences = []
words = []
sentences3 = []
sentences5 = []
id_dict = {key:[] for key in ['id_dokument','id_fall', 'fall_source_key', 'id_k', 'name', 'vorname',]}
for ii, row in df.iterrows():
    pos_words, pos_sentences, pos_sentences3, pos_sentences5 = ViolenceFinder3(row['text']).pos_sentences()
    
    sentences += pos_sentences
#     pos_sentences3 = ViolenceFinder(row['text']).pos_sentences3()
    sentences3 += pos_sentences3
#     print(pos_sentences3, len(pos_sentences3), pos_sentences, len(pos_sentences))
    sentences5 += pos_sentences5
    words +=pos_words
    for key, el in id_dict.items():
        el += [row[key]]*len(pos_sentences)
    
sent_df = pd.DataFrame()
for key, el in id_dict.items():
    sent_df[key] = el
sent_df['Nsatz'] = sent_df.groupby(['id_dokument'])[['id_dokument']].count()['id_dokument'].reindex(sent_df['id_dokument']).values
sent_df['Satz'] = sentences
sent_df['pos_words'] = words
sent_df['Satz3'] = sentences3
sent_df['Satz5'] = sentences5
print('Anzahl Sätze:',sent_df.shape[0])
print('Anzahl Dokumente:',len(sent_df['id_dokument'].unique()))
sent_df.head()


#count doubles in sent_df columns
print('Anzahl doppelter Sätze:',sent_df['Satz'].duplicated().sum())
#filter out doubles in sent_df columns
sent_df2 = sent_df.drop_duplicates(subset=['Satz'])
print('Anzahl Sätze nach Filter:',sent_df2.shape[0])
#check if there are still doubles
print('Anzahl doppelter Sätze nach Filter:',sent_df2['Satz'].duplicated().sum())
sent3_df = sent_df2.copy()

Anzahl Sätze: 15605
Anzahl Dokumente: 7542
Anzahl doppelter Sätze: 2203
Anzahl Sätze nach Filter: 13402
Anzahl doppelter Sätze nach Filter: 0


In [7]:
# create a list with stemmed words of the list RE_POS_STEMS
RE_POS_STEMS = [
  ' Angriff ',
        ' angreifen ',
    ' aggressiv ',
    ' Aggression ',
    ' Aggresivität ',    
' anschreien ',
' aufdringlich ',
# ' Auge ',
# ' ausschliessen ',
' bedrohen ',
' begrabscht ',
' Belästigen ',
' beleidigen ',
' berührt ',
' beschimpfen ',
' Bestraffung ',
' blaue ',
' blutige ',
# ' eingeführt ',
' Castagna ',
' einschüchtern ',
' einsperren ',
' erniedrigen ',
' Faust ',
' Flecken ',
' gebrochen ',
' gefährden ',
' Gefährdung ',
' geküsst ',
' Genitalien ',
# ' gezwungen ',
' kneifen ',
' Knochen ',
' körperlich ',
' Kratzer ',
' Kindeswohl ',
' Kindeswohlgefährdung ',
' lächerlich ',
' Lippen ',
' Misshandlung ',
' misshandeln ',
' missbrauchen ',
' Missbrauch ',
' nackt ',
' Nase ',
' ohrfeigen ',
' packen ',
' Penis ',
' Po ',
' Polizei ',
' polizeilich ',
' Polizeirapport ',
' Polizeibericht ',
' Prellung ',
' Quetschung ',
' rote ',
' Scheide ',
' Schnittwunde ',
' schubsen ',
' schütteln ',
' sexuell ',
' stossen ',
' Strafe ',
' Strafverfahren ',
# ' treten ',
' Vagina ',
' Verbrennung ',
' Verletzung ',
' Verstauchung ',
' Vorfall ',
' Waffe ',
' Weh ',
' werfen ',
' würgen ',
]


stemmer = SnowballStemmer("german")
stemmed_words = [stemmer.stem(word) for word in RE_POS_STEMS]
stemmed_words = list(set(stemmed_words))
stemmed_words.sort()
print(stemmed_words)
#add for every list item string '\w'





[' aggresivitat ', ' aggression ', ' aggressiv ', ' angreifen ', ' angriff ', ' anschreien ', ' aufdringlich ', ' bedrohen ', ' begrabscht ', ' belastigen ', ' beleidigen ', ' beruhrt ', ' beschimpfen ', ' bestraffung ', ' blaue ', ' blutige ', ' castagna ', ' einschuchtern ', ' einsperren ', ' erniedrigen ', ' faust ', ' flecken ', ' gebrochen ', ' gefahrden ', ' gefahrdung ', ' gekusst ', ' genitalien ', ' kindeswohl ', ' kindeswohlgefahrdung ', ' kneifen ', ' knochen ', ' korperlich ', ' kratzer ', ' lacherlich ', ' lippen ', ' missbrauch ', ' missbrauchen ', ' misshandeln ', ' misshandlung ', ' nackt ', ' nase ', ' ohrfeigen ', ' packen ', ' penis ', ' po ', ' polizei ', ' polizeibericht ', ' polizeilich ', ' polizeirapport ', ' prellung ', ' quetschung ', ' rote ', ' scheide ', ' schnittwunde ', ' schubsen ', ' schutteln ', ' sexuell ', ' stossen ', ' strafe ', ' strafverfahren ', ' vagina ', ' verbrennung ', ' verletzung ', ' verstauchung ', ' vorfall ', ' waffe ', ' weh ', ' wer

In [8]:
print(RE_POS_STEMS)

[' Angriff ', ' angreifen ', ' aggressiv ', ' Aggression ', ' Aggresivität ', ' anschreien ', ' aufdringlich ', ' bedrohen ', ' begrabscht ', ' Belästigen ', ' beleidigen ', ' berührt ', ' beschimpfen ', ' Bestraffung ', ' blaue ', ' blutige ', ' Castagna ', ' einschüchtern ', ' einsperren ', ' erniedrigen ', ' Faust ', ' Flecken ', ' gebrochen ', ' gefährden ', ' Gefährdung ', ' geküsst ', ' Genitalien ', ' kneifen ', ' Knochen ', ' körperlich ', ' Kratzer ', ' Kindeswohl ', ' Kindeswohlgefährdung ', ' lächerlich ', ' Lippen ', ' Misshandlung ', ' misshandeln ', ' missbrauchen ', ' Missbrauch ', ' nackt ', ' Nase ', ' ohrfeigen ', ' packen ', ' Penis ', ' Po ', ' Polizei ', ' polizeilich ', ' Polizeirapport ', ' Polizeibericht ', ' Prellung ', ' Quetschung ', ' rote ', ' Scheide ', ' Schnittwunde ', ' schubsen ', ' schütteln ', ' sexuell ', ' stossen ', ' Strafe ', ' Strafverfahren ', ' Vagina ', ' Verbrennung ', ' Verletzung ', ' Verstauchung ', ' Vorfall ', ' Waffe ', ' Weh ', ' wer

In [9]:
#liste wurde korrigiert, die meisten Begriffe haben \w erhalten damit weitere Endungen auch gefunden weden, eizelne Substantive wie Faust müssen als ganze Wörter gefunden werden deshalb ' Faust '
stem_list=[' aggresivitat\w', ' aggression\w', ' aggressiv\w', ' angreifen\w', ' angriff\w', ' anschreien\w', ' aufdringlich\w', ' bedrohen\w', ' begrabscht\w', ' belastigen\w', ' beleidigen\w', ' beruhrt\w', ' beschimpfen\w', ' bestraffung\w', ' blaue\w', ' blutige\w', ' castagna\w', ' einschuchtern\w', ' einsperren\w', ' erniedrigen\w', ' faust', ' flecken\w', ' gebrochen\w', ' gefahrden\w', ' gefahrdung\w', ' gekusst\w', ' genitalien\w', ' kindeswohl\w', ' kindeswohlgefahrdung\w', ' kneifen\w', ' knochen\w', ' korperlich\w', ' kratzer\w', ' lacherlich\w', ' lippen\w', ' missbrauch\w', ' missbrauchen\w', ' misshandeln\w', ' misshandlung\w', ' nackt\w', ' nase ', ' ohrfeigen\w', ' packen\w', ' penis\w', ' po ', ' polizei\w', ' polizeibericht\w', ' polizeilich\w', ' polizeirapport\w', ' prellung\w', ' quetschung\w', ' rote\w', '  scheide', ' schnittwunde\w', ' schubsen\w', ' schutteln\w', ' sexuell\w', ' stossen\w', ' strafe\w', ' strafverfahren\w', ' vagina\w', ' verbrennung\w', ' verletzung\w', ' verstauchung\w', ' vorfall\w', ' waffe\w', ' weh ', ' werfen\w', ' wurgen\w',
'Angriff', 'angreifen', 'aggressiv', 'Aggression', 'Aggresivität', 'anschreien', 'aufdringlich', 'bedrohen', 'begrabscht', 'Belästigen', 'beleidigen', 'berührt', 'beschimpfen', 'Bestraffung',  'blutige', 'Castagna', 'einschüchtern', 'einsperren', 'erniedrigen', ' Faust', ' Flecken', ' gebrochen', 'gefährden', 'Gefährdung', 'Genitalien', 'kneifen', 'Knochen', 'körperlich', 'Kratzer', 'Kindeswohl', 'Kindeswohlgefährdung', 'lächerlich',  'Misshandlung', 'misshandeln', 'missbrauchen', 'Missbrauch', 'nackt', ' Nase ', 'ohrfeigen', 'packen', ' Penis',  'Polizei', 'polizeilich', 'Polizeirapport', 'Polizeibericht', 'Prellung', 'Quetschung', ' Scheide ', 'Schnittwunde', 'schubsen', 'schütteln', 'sexuell', 'stossen', 'Strafe', 'Strafverfahren', 'Vagina', 'Verbrennung', 'Verletzung', 'Verstauchung', 'Vorfall', 'Waffe', ' Weh ', 'werfen', 'würgen ']
# ['aggresivitat\w', 'aggression\w', 'aggressiv\w', 'angreifen\w', 'angriff\w', 'anschreien\w', 'aufdringlich\w', 'bedrohen\w', 'begrabscht\w', 'belastigen\w', 'beleidigen\w', 'beruhrt\w', 'beschimpfen\w', 'bestraffung\w', 'blaue\w', 'blutige\w', 'castagna\w', 'einschuchtern\w', 'einsperren\w', 'erniedrigen\w', 'faust ', 'flecken\w', 'gebrochen\w', 'gefahrden\w', 'gefahrdung\w', 'gekusst\w', 'genitalien\w', 'kindeswohl\w', 'kindeswohlgefahrdung\w', 'kneifen\w', 'knochen\w', 'korperlich\w', 'kratzer\w', 'lacherlich\w', 'lippen\w', 'missbrauch\w', 'missbrauchen\w', 'misshandeln\w', 'misshandlung\w', 'nackt\w', 'nase ', 'ohrfeigen\w', 'packen\w', 'penis\w', 'po ', 'polizei\w', 'polizeibericht\w', 'polizeilich\w', 'polizeirapport\w', 'prellung\w', 'quetschung\w', 'rote\w', 'scheide ', 'schnittwunde\w', 'schubsen\w', 'schutteln\w', 'sexuell\w', 'stossen\w', 'strafe\w', 'strafverfahren\w', 'vagina\w', 'verbrennung\w', 'verletzung\w', 'verstauchung\w', 'vorfall\w', 'waffe\w', 'weh ', 'werfen\w', 'wurgen\w',]
# [' aggresivitat\w', ' aggression\w', ' aggressiv\w', ' angreifen\w', ' angriff\w', ' anschreien\w', ' aufdringlich\w', ' bedrohen\w', ' begrabscht\w', ' belastigen\w', ' beleidigen\w', ' beruhrt\w', ' beschimpfen\w', ' bestraffung\w', ' blaue\w', ' blutige\w', ' castagna\w', ' einschuchtern\w', ' einsperren\w', ' erniedrigen\w', ' faust ', ' flecken\w', ' gebrochen\w', ' gefahrden\w', ' gefahrdung\w', ' gekusst\w', ' genitalien\w', ' kindeswohl\w', ' kindeswohlgefahrdung\w', ' kneifen\w', ' knochen\w', ' korperlich\w', ' kratzer\w', ' lacherlich\w', ' lippen\w', ' missbrauch\w', ' missbrauchen\w', ' misshandeln\w', ' misshandlung\w', ' nackt\w', ' nase ', ' ohrfeigen\w', ' packen\w', ' penis\w', ' po ', ' polizei\w', ' polizeibericht\w', ' polizeilich\w', ' polizeirapport\w', ' prellung\w', ' quetschung\w', ' rote\w', ' scheide ', ' schnittwunde\w', ' schubsen\w', ' schutteln\w', ' sexuell\w', ' stossen\w', ' strafe\w', ' strafverfahren\w', ' vagina\w', ' verbrennung\w', ' verletzung\w', ' verstauchung\w', ' vorfall\w', ' waffe\w', ' weh ', ' werfen\w', ' wurgen\w',]

class ViolenceFinderNeue:
    RE_POS_STEMS = stem_list

    FALSE_POS_STEMS = ['schlagmüller','ausschlag','einschlag','eingeschlag','niederschlag','niedergeschlag',
                      'schicksalsschlag','angeschlag','vorschlag','vorgeschlag','vorschläge','voranschlag',
                      'ratschlag','ratschläge','schlagzeug','schleg'
                                            'abgetreten', 
'abtreten',
'abgetreten', 
'abtretenden', 
'abzutreten', 
'angetreten', 
'antreten', 
'anzupacken', 
'anzustossen', 
'angestossen',
'anzutreten', 
'aufgetreten', 
'aufgetretene', 
'aufgetretenen', 
'aufgezwungen', 
'auftreten', 
'auftretende', 
'auftretenden', 
'auftretender', 
'auftretens', 
'aufwerfen', 
'aufzutreten', 
'ausgetreten', 
'austreten', 
'auszutreten', 
'beigetreten', 
'beitreten', 
'beizutreten', 
'betreten', 
'eingetreten', 
'eingetretene', 
'eingetretenen', 
'einpacken', 
'eintreten', 
'einzupacken', 
'einzutreten', 
'entgegentreten', 
'entgegenzutreten', 
'entwerfen', 
'gegenübertreten', 
'gezwungenermassen', 
'herangetreten', 
'heraustreten', 
'inkrafttreten', 
'nichteintreten', 
'nichteintretensentscheid', 
'protect', 
'protection', 
'protest', 
'rotegg', 
'spaccarotella', 
'stellvertreten', 
'stellvertretend', 
'stellvertretende', 
'stellvertretenden', 
'stellvertretender', 
'übergetreten', 
'überzutreten', 
'ungezwungen', 
'ungezwungenen', 
'ungezwungener', 
'vertreten', 
'vertreten', 'vertreten', 
'vertretende', 
'vertretenden', 
'wegwerfen', 
'anpackende', 
'beinprotese', 
'blauenstrasse', 
'blauenweg', 
'brote', 
'broteinheiten', 
'brotes', 
'bürotermine', 
'dazugestossen', 
'elektrotech', 
'elektrotechnik', 
'elektrotechniker', 
'elektrotechnischen', 
'konktreten', 
'kürzertreten', 
'nachaussentreten', 
'protezione', 
'rotenbach', 
'rotenberg', 
'rotenbühler', 'rotenbühler', 
'rotenburger', 
'rotenflueh', 
'rotenfluh', 
'saroten', 
'schroten', 
'tourotel', 
'übertreten', 
'unberührt', 
'verkörperlichung', 
'vetreten', 
'zurückgetreten', 
'zurücktreten', 
'zusammenpacken', 
]
    CUT_PHRASES = ['(Welche Lösungen bzw. welches Vorgehen schlägt das Kind bzw. der/die Jugendliche vor)']
    SPLITCHAR = '.'
    SPLITPHRASES = ['Ausgangslage, Auftrag und Ziele\n','Situation, Entwicklung\n','Auftrag und Ziele\n',
                   'Vorgeschichte\n']
    SPLITPATTERN = r'((?<!(z\.B|.\s\w|.\W\w|..\d|Art|bzw|geb|Abs|Stv|etc))\.|;|:|\n\w\)|--|\n\n|\d*\.\d*\.\d\d\d\d\n|\{})'.format('|'.join(SPLITPHRASES))     # three characte expressions
    
    def __init__(self, text):
        self.text = text
        self.sentences = [s for s in re.split(self.SPLITPATTERN, text) if s is not None]
        self.pos_words = [self._find_pos_words(s) for s in self.sentences]
        self.labels = [len(words)>0 for words in self.pos_words]
                
    def _find_pos_words(self,text):
        for phrase in self.CUT_PHRASES:
            text = text.lower().replace(phrase.lower(),'')
        pattern = [r'\b\w*{}\w*\b'.format(stem) for stem in self.RE_POS_STEMS]
        pattern = r'(' + r'|'.join(pattern) + r')'
        # print(pattern)
        pos_words = []
        for word in re.findall(pattern, text.lower()):
            if not np.any([fpword in word for fpword in self.FALSE_POS_STEMS]):
                pos_words.append(word)
                # print("word:", word)
                # print("satz:", text)
        return pos_words
    
    def label(self):
        return np.any(self.labels)
        
    def pos_sentences(self):
        out = []
        words0 = []
        for ind in np.where(self.labels)[0]:
            out.append(self.sentences[ind].replace('\n','').strip()+self.SPLITCHAR)
            words0.append(self.pos_words[ind])
        
        out3 = []
        out5 = []
        
        for ind in np.where(self.labels)[0]:
            try:
                tempM1 = (self.sentences[ind-2].replace('\n','').strip()+self.SPLITCHAR+' ')
            except IndexError:
                tempM1 = self.SPLITCHAR
            
            temp = (self.sentences[ind].replace('\n','').strip()+self.SPLITCHAR+' ')
            
            try:
                tempP1 =(self.sentences[ind+2].replace('\n','').strip()+self.SPLITCHAR)
            except IndexError:
                tempP1 = self.SPLITCHAR
            
            try:
                tempM2 = (self.sentences[ind-4].replace('\n','').strip()+self.SPLITCHAR+' ')
            except IndexError:
                tempM2 = self.SPLITCHAR
            try:
                tempP2 =(self.sentences[ind+4].replace('\n','').strip()+self.SPLITCHAR)
            except IndexError:
                tempP2 = self.SPLITCHAR 
                
            out3_temp = tempM1 + temp + tempP1
            out3.append(out3_temp)
        #   print(out3_temp)
            out5_temp = tempM2 + tempM1 + temp + tempP1 + tempP2
            out5.append(out5_temp)
            
        return words0, out, out3, out5
        
    
    def marked_text(self, ansi='\033[44;33m', nul = "\033[m"):
        text = ''
        for ii, s in enumerate(self.sentences):
            if self.labels[ii]:
                text += ansi + s + nul + self.SPLITCHAR
            else:
                text += s + self.SPLITCHAR
        return text
    

    def pprint(self):
        print(self.marked_text())


In [10]:
#print 50 random rows with columns 'Satz' and 'pos_words'
# sent3_df[['Satz','pos_words']].sample(50)

In [11]:
sentences = []
words = []
sentences3 = []
sentences5 = []
id_dict = {key:[] for key in ['id_dokument','id_fall', 'fall_source_key', 'id_k', 'name', 'vorname', 'name_mutter', 'vorname_mutter', 'name_vater', 'vorname_vater',]}
for ii, row in df.iterrows():
    pos_words, pos_sentences, pos_sentences3, pos_sentences5 = ViolenceFinderNeue(row['text']).pos_sentences()
    
    sentences += pos_sentences
#     pos_sentences3 = ViolenceFinder(row['text']).pos_sentences3()
    sentences3 += pos_sentences3
#     print(pos_sentences3, len(pos_sentences3), pos_sentences, len(pos_sentences))
    sentences5 += pos_sentences5
    words +=pos_words
    for key, el in id_dict.items():
        el += [row[key]]*len(pos_sentences)
    
sent_df = pd.DataFrame()
for key, el in id_dict.items():
    sent_df[key] = el
sent_df['Nsatz'] = sent_df.groupby(['id_dokument'])[['id_dokument']].count()['id_dokument'].reindex(sent_df['id_dokument']).values
sent_df['Satz'] = sentences
sent_df['pos_words'] = words
sent_df['Satz3'] = sentences3
sent_df['Satz5'] = sentences5
print('Anzahl Sätze:',sent_df.shape[0])
print('Anzahl Dokumente:',len(sent_df['id_dokument'].unique()))
sent_df.head()
#count doubles in sent_df columns
print('Anzahl doppelter Sätze:',sent_df['Satz'].duplicated().sum())
#filter out doubles in sent_df columns
sent_df2 = sent_df.drop_duplicates(subset=['Satz'])
print('Anzahl Sätze nach Filter:',sent_df2.shape[0])
#check if there are still doubles
print('Anzahl doppelter Sätze nach Filter:',sent_df2['Satz'].duplicated().sum())
sentNeue_df = sent_df2.copy()
# print(sentNeue_df.head())

Anzahl Sätze: 21584
Anzahl Dokumente: 11390
Anzahl doppelter Sätze: 2762
Anzahl Sätze nach Filter: 18822
Anzahl doppelter Sätze nach Filter: 0


In [12]:
#count values of column 'pos_words' and show all of them in output
freq_liest=sentNeue_df['pos_words'].value_counts()
freq_liest=pd.DataFrame(freq_liest)
print(freq_liest.head(100))
print(len(freq_liest))
#save freq_liest to excel
# freq_liest.to_excel('freq_liest.xlsx')

                               pos_words
[körperlich]                        1448
[aggressiv]                         1395
[körperliche]                       1347
[körperlichen]                       950
[des kindeswohls]                    824
[gefährden]                          730
[körperlicher]                       304
[stossen]                            182
[des strafverfahrens]                153
[gestossen]                          149
[werfen]                             135
[einem polizeieinsatz]               128
[des kindeswohles]                   113
[sexuell]                            105
[packen]                              94
[des vorfalls]                        94
[ein aggressives]                     87
[und verletzungen]                    86
[verstossen]                          86
[die verletzungen]                    81
[gefährdend]                          80
[aggressives]                         70
[eine kindeswohlgefährdung]           69
[mit aggressivem

In [13]:
#join sent3_df and sentNeue_df based on Satz column and keep only rows that are not in sent3_df and save it as sentNeueOhne3_df
sentNeueOhne3_df = sentNeue_df[~sentNeue_df['Satz'].isin(sent3_df['Satz'])]
print('Anzahl Sätze in sentNeueOhne3_df:',sentNeueOhne3_df.shape[0])
#drop rows that contain 'in Kontakt' in Satz column and and contain 'treten' in Satz column and save it in new df sentNeueClean1_df
sentNeueClean1_df = sentNeueOhne3_df[~((sentNeueOhne3_df['Satz'].str.contains('treten')) & (sentNeueOhne3_df['Satz'].str.contains('in Kontakt')))]
sentNeueClean2_df = sentNeueClean1_df[~((sentNeueClean1_df['Satz'].str.contains('[Kk]örp' )) & (sentNeueClean1_df['Satz'].str.contains('[Gg]esun' )))]
sentNeueClean3_df = sentNeueClean2_df[~((sentNeueClean2_df['Satz'].str.contains('[Kk]örp' )) & (sentNeueClean2_df['Satz'].str.contains('[Ee]ntwick' )))]
sentNeueClean4_df = sentNeueClean3_df[~((sentNeueClean3_df['Satz'].str.contains('[Gg]renz' )) & (sentNeueClean3_df['Satz'].str.contains('[Ss]toss' )))]
sentNeueClean5_df = sentNeueClean4_df[~((sentNeueClean4_df['Satz'].str.contains('vor den Kopf')) & (sentNeueClean4_df['Satz'].str.contains('stoss' )))]

sentNeueClean6_df = sentNeueClean5_df[~((sentNeueClean5_df['Satz'].str.contains('[Rr]ot' )) & (sentNeueClean5_df['Satz'].str.contains('[Kk]reuz')))]

sentNeueClean7_df = sentNeueClean6_df[~((sentNeueClean6_df['Satz'].str.contains('[Rr]ot' )) & (sentNeueClean6_df['Satz'].str.contains('Faden')))]
print('Anzahl Sätze in sentNeueClean1_df:',sentNeueClean1_df.shape[0])
print('Anzahl Sätze in sentNeueClean2_df:',sentNeueClean2_df.shape[0])
print('Anzahl Sätze in sentNeueClean3_df:',sentNeueClean3_df.shape[0])
print('Anzahl Sätze in sentNeueClean4_df:',sentNeueClean4_df.shape[0])
print('Anzahl Sätze in sentNeueClean5_df:',sentNeueClean5_df.shape[0])
print('Anzahl Sätze in sentNeueClean6_df:',sentNeueClean6_df.shape[0])
print('Anzahl Sätze in sentNeueClean7_df:',sentNeueClean7_df.shape[0])
#save sentNeueClean7_df as csv and as pickle and as excel
# sentNeueClean7_df.to_csv('sentNeueClean7aggr2_df.csv', index=False)
# sentNeueClean7_df.to_pickle('sentNeueClean7aggr2_df.pkl')
# sentNeueClean7_df.to_excel('sentNeueClean7aggr2_df.xlsx')



Anzahl Sätze in sentNeueOhne3_df: 16930
Anzahl Sätze in sentNeueClean1_df: 16922
Anzahl Sätze in sentNeueClean2_df: 16073
Anzahl Sätze in sentNeueClean3_df: 15155
Anzahl Sätze in sentNeueClean4_df: 14976
Anzahl Sätze in sentNeueClean5_df: 14956
Anzahl Sätze in sentNeueClean6_df: 14838
Anzahl Sätze in sentNeueClean7_df: 14821


In [15]:
# conda install -c conda-forge pypdf2
# conda install nltk 
# pip install HanTa

import re, nltk, os, glob, docx, json, gensim
import pandas as pd
import numpy as np
import seaborn as sns
import wehs_helpers as wh
# from HanTa import HanoverTagger as ht
from docx2python import docx2python
import win32com.client
from bs4 import BeautifulSoup as bs
import gensim.corpora as corpora
import spacy

# import stopwords
# nltk.download('stopwords')  
os.name  # nt means windows
pd.options.display.max_colwidth = 1000
nlp = spacy.load("de_core_news_lg")
pd.set_option('display.max_rows', 500)
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer 
stemmer = SnowballStemmer("german")
import pickle
pd.options.display.max_colwidth = 1000

In [16]:
from pathlib import Path    
saetze_3Begriffe = sent3_df
# pd.read_pickle('GewaltSaetzeProzessiert_20230212.pkl')
#change alle column names to lowercase
saetze_3Begriffe.columns = map(str.lower, saetze_3Begriffe.columns)
saetze_3Begriffe.columns


Index(['id_dokument', 'id_fall', 'fall_source_key', 'id_k', 'name', 'vorname',
       'nsatz', 'satz', 'pos_words', 'satz3', 'satz5'],
      dtype='object')

In [17]:

#keep only the columns 'id_dokument', 'id_k', 'nsatz', 'satz', 'satz3', 'satz5', 'name', 'vorname'
# saetze_3Begriffe = saetze_3Begriffe[['id_dokument', 'id_k', 'nsatz', 'satz', 'satz3', 'satz5', 'name', 'vorname']]
#sort columns in dataframe by alphabetical order
# saetze_3Begriffe = saetze_3Begriffe.reindex(sorted(saetze_3Begriffe.columns), axis=1)
# saetze_3Begriffe.columns
#create new column datensatz with value 'drei Begriffe'
saetze_3Begriffe['datensatz'] = 'drei Begriffe'

In [20]:
#read excel file sentNeueClean7joined2_df.xlsx and create dataframe
saetze_weiterBegriffe = sentNeueClean7_df
# pd.read_excel('sentNeueClean7joined2_df.xlsx', sheet_name='Sheet1')


#change alle column names to lowercase
saetze_weiterBegriffe.columns = map(str.lower, saetze_weiterBegriffe.columns)
#drop columns    'name_mutter', 'vorname_mutter', 'name_vater', 'vorname_vater',
saetze_weiterBegriffe = saetze_weiterBegriffe.drop(['name_mutter', 'vorname_mutter', 'name_vater', 'vorname_vater'], axis=1)
saetze_weiterBegriffe.columns

Index(['id_dokument', 'id_fall', 'fall_source_key', 'id_k', 'name', 'vorname',
       'nsatz', 'satz', 'pos_words', 'satz3', 'satz5'],
      dtype='object')

In [21]:

#keep only the columns 'fall_source_key', 'id_dokument', 'id_fall', 'id_k', 'name','nsatz', 'pos_words', 'satz', 'satz3', 'satz5', 'vorname'
# saetze_weiterBegriffe = saetze_weiterBegriffe[['fall_source_key', 'id_dokument', 'id_fall', 'id_k', 'name','nsatz', 'pos_words', 'satz', 'satz3', 'satz5', 'vorname']]
#sort columns in dataframe by alphabetical order
# saetze_weiterBegriffe = saetze_weiterBegriffe.reindex(sorted(saetze_weiterBegriffe.columns), axis=1)
# saetze_weiterBegriffe.columns
#create new column datensatz with value 'weitere Begriffe'
saetze_weiterBegriffe['datensatz'] = 'weitere Begriffe'

In [37]:
#join to dataframes saetze_3Begriffe and saetze_weiterBegriffe
saetze_alleBegriffe = pd.concat([saetze_3Begriffe, saetze_weiterBegriffe], axis=0, ignore_index=True)



# replace the '|' with space in column 'satz5'
# saetze_alleBegriffe['satz5'] = saetze_alleBegriffe['satz5'].str.replace('|', ' ')
#replace four spaces with one space in column 'satz5'
saetze_alleBegriffe['satz5'] = saetze_alleBegriffe['satz5'].str.replace('    ', ' ')
#replace three spaces with one space in column 'satz5'
saetze_alleBegriffe['satz5'] = saetze_alleBegriffe['satz5'].str.replace('   ', ' ')
#replace two spaces with one space in column 'satz5'
saetze_alleBegriffe['satz5'] = saetze_alleBegriffe['satz5'].str.replace('  ', ' ')

#remove all spaces and tabs in column 'satz5' and save it in new column 'satz5_ohne_leerzeichen'
saetze_alleBegriffe['satz5_ohne_leerzeichen'] = saetze_alleBegriffe['satz5'].str.replace(' ', '')
saetze_alleBegriffe['satz5_ohne_leerzeichen'] = saetze_alleBegriffe['satz5_ohne_leerzeichen'].str.replace('\t', '')
#show all columns and missing values
saetze_alleBegriffe.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28223 entries, 0 to 28222
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id_dokument             28223 non-null  int64 
 1   id_fall                 28223 non-null  int64 
 2   fall_source_key         28223 non-null  object
 3   id_k                    28223 non-null  int64 
 4   name                    28223 non-null  object
 5   vorname                 28223 non-null  object
 6   nsatz                   28223 non-null  int64 
 7   satz                    28223 non-null  object
 8   pos_words               28223 non-null  object
 9   satz3                   28223 non-null  object
 10  satz5                   28223 non-null  object
 11  datensatz               28223 non-null  object
 12  satz5_ohne_leerzeichen  28223 non-null  object
dtypes: int64(4), object(9)
memory usage: 2.8+ MB


<ipython-input-37-e922c20f2419>:19: FutureWarning: null_counts is deprecated. Use show_counts instead
  saetze_alleBegriffe.info(verbose=True, null_counts=True)


In [40]:
#load excel table train_data.xlsx into dataframe train_data
train_data = pd.read_excel('train_data.xlsx', sheet_name='Sheet1')
#load excel table test_data.xlsx into dataframe test_data
test_data = pd.read_excel('test_data.xlsx', sheet_name='Sheet1')
#join dataframes train_data and test_data
train_test_data = pd.concat([train_data, test_data], axis=0, ignore_index=True)
#show all columns and missing values
train_test_data.info(verbose=True, null_counts=True)
#count label values
print(train_test_data['label'].value_counts())
#recode label values 0 to 'physiche Gewalt', 1 to 'häusliche Gewalt', 2 to 'ausgeübte Gewalt', 3 to 'sexuelle Gewalt' and 4 to 'keine Gewalt' in a new column 'annotiert_string'
train_test_data['annotiert_string'] = train_test_data['label'].replace({0: 'physiche Gewalt', 1: 'häusliche Gewalt', 2: 'ausgeübte Gewalt', 3: 'sexuelle Gewalt', 4: 'keine Gewalt'})
#count annotiert_string values
train_test_data['annotiert_string'].value_counts()




#rename column label to annotiert and senctence to satz5
train_test_data.rename(columns={'label': 'annotiert', 'sentence': 'satz5'}, inplace=True)

#replace four spaces with one space in column 'satz5'
train_test_data['satz5'] = train_test_data['satz5'].str.replace('    ', ' ')
#replace three spaces with one space in column 'satz5'
train_test_data['satz5'] = train_test_data['satz5'].str.replace('   ', ' ')
#replace two spaces with one space in column 'satz5'
train_test_data['satz5'] = train_test_data['satz5'].str.replace('  ', ' ')
train_test_data['satz5'] = train_test_data['satz5'].str.replace('  ', ' ')
train_test_data['satz5'] = train_test_data['satz5'].str.replace('  ', ' ')
train_test_data['satz5'] = train_test_data['satz5'].str.replace('  ', ' ')
#remove all spaces and tabs in column 'satz5' and save it in new column 'satz5_ohne_leerzeichen'
train_test_data['satz5_ohne_leerzeichen'] = train_test_data['satz5'].str.replace(' ', '')

train_test_data['satz5_ohne_leerzeichen'] = train_test_data['satz5_ohne_leerzeichen'].str.replace('\t', '')

#rename column 'satz5' to 'satz5_annotiert'
train_test_data.rename(columns={'satz5': 'satz5_annotiert'}, inplace=True)
#drop column Unnamed: 0
train_test_data.drop(columns=['Unnamed: 0'], inplace=True)
print(train_test_data.info(verbose=True, null_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4397 entries, 0 to 4396
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4397 non-null   int64 
 1   label       4397 non-null   int64 
 2   sentence    4396 non-null   object
dtypes: int64(2), object(1)
memory usage: 103.2+ KB
4    1328
1    1036
0     876
2     872
3     285
Name: label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4397 entries, 0 to 4396
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   annotiert               4397 non-null   int64 
 1   satz5_annotiert         4396 non-null   object
 2   annotiert_string        4397 non-null   object
 3   satz5_ohne_leerzeichen  4396 non-null   object
dtypes: int64(1), object(3)
memory usage: 137.5+ KB
None


<ipython-input-40-d94646a2f40a>:8: FutureWarning: null_counts is deprecated. Use show_counts instead
  train_test_data.info(verbose=True, null_counts=True)
<ipython-input-40-d94646a2f40a>:40: FutureWarning: null_counts is deprecated. Use show_counts instead
  print(train_test_data.info(verbose=True, null_counts=True))


In [41]:
#join dataframes saetze_alleBegriffe and train_test_data on column 'satz5' and save it in dataframe saetze_alleBegriffe_annotiert
saetze_alleBegriffe_annotiert = pd.merge(saetze_alleBegriffe, train_test_data, on='satz5_ohne_leerzeichen', how='left')
#show all columns and missing values
saetze_alleBegriffe_annotiert.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28223 entries, 0 to 28222
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_dokument             28223 non-null  int64  
 1   id_fall                 28223 non-null  int64  
 2   fall_source_key         28223 non-null  object 
 3   id_k                    28223 non-null  int64  
 4   name                    28223 non-null  object 
 5   vorname                 28223 non-null  object 
 6   nsatz                   28223 non-null  int64  
 7   satz                    28223 non-null  object 
 8   pos_words               28223 non-null  object 
 9   satz3                   28223 non-null  object 
 10  satz5                   28223 non-null  object 
 11  datensatz               28223 non-null  object 
 12  satz5_ohne_leerzeichen  28223 non-null  object 
 13  annotiert               4396 non-null   float64
 14  satz5_annotiert         4396 non-null 

<ipython-input-41-02009c8226e9>:4: FutureWarning: null_counts is deprecated. Use show_counts instead
  saetze_alleBegriffe_annotiert.info(verbose=True, null_counts=True)


In [25]:
# #show me an example of fuzzy matching to dataframes 
# # !pip install fuzzy_pandas

# #drop rows with missing values in column satz5 in both dataframes
# saetze_alleBegriffe.dropna(subset=['satz5'], inplace=True)
# train_test_data.dropna(subset=['satz5'], inplace=True)
# #show all columns and missing values
# import fuzzy_pandas as fpd


# saetze_alleBegriffe_annotiert = fpd.fuzzy_merge(saetze_alleBegriffe, train_test_data,
#             left_on='satz5',
#             right_on='satz5',
#             method='levenshtein',
#             threshold=0.9)


In [ ]:

saetze_alleBegriffe_annotiert.head()




In [42]:
#print all columns and missing values
saetze_alleBegriffe_annotiert.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28223 entries, 0 to 28222
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_dokument             28223 non-null  int64  
 1   id_fall                 28223 non-null  int64  
 2   fall_source_key         28223 non-null  object 
 3   id_k                    28223 non-null  int64  
 4   name                    28223 non-null  object 
 5   vorname                 28223 non-null  object 
 6   nsatz                   28223 non-null  int64  
 7   satz                    28223 non-null  object 
 8   pos_words               28223 non-null  object 
 9   satz3                   28223 non-null  object 
 10  satz5                   28223 non-null  object 
 11  datensatz               28223 non-null  object 
 12  satz5_ohne_leerzeichen  28223 non-null  object 
 13  annotiert               4396 non-null   float64
 14  satz5_annotiert         4396 non-null 

<ipython-input-42-72d40aae49b5>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  saetze_alleBegriffe_annotiert.info(verbose=True, null_counts=True)


In [43]:
#create crosstab with datensatz and annotiert_string
pd.crosstab(saetze_alleBegriffe_annotiert['datensatz'], saetze_alleBegriffe_annotiert['annotiert_string'], margins=True)

annotiert_string,ausgeübte Gewalt,häusliche Gewalt,keine Gewalt,physiche Gewalt,sexuelle Gewalt,All
datensatz,,,,,,
drei Begriffe,680,939,816,787,187,3409
weitere Begriffe,192,96,512,89,98,987
All,872,1035,1328,876,285,4396


In [ ]:
#save saetze_alleBegriffe_annotiert as csv and as pickle and as excel with name  saetze_alleBegriffe_annotiert_20230829
saetze_alleBegriffe_annotiert.to_csv('saetze_alleBegriffe_annotiert_20230829.csv', index=False)
saetze_alleBegriffe_annotiert.to_pickle('saetze_alleBegriffe_annotiert_20230829.pkl')
saetze_alleBegriffe_annotiert.to_excel('saetze_alleBegriffe_annotiert_20230829.xlsx')